In [1]:
import keras.backend as K
import math
import numpy as np
import h5py
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
def mean_pred(y_true, y_pred):
    return K.mean(y_pred)

In [3]:
def load_dataset():
    train_dataset = h5py.File('data/train.h5','r')
    train_set_x = np.array(train_dataset['train_set_x'][:])
    train_set_y = np.array(train_dataset['train_set_y'][:])
    
    test_dataset = h5py.File('data/test.h5','r')
    test_set_x = np.array(test_dataset['test_set_x'][:])
    test_set_y = np.array(test_dataset['test_set_y'][:])
    
    classes = np.array(test_dataset['list_classes'][:])
    
    #train_set_x = train_set_x.reshape((1, train_set_x.shape[0])) 
    #test_set_x = test_set_x.reshape((1, test_set_x.shape[0]))
    
    return train_set_x,train_set_y,test_set_x,test_set_y,classes

In [4]:
import numpy as np
from keras import layers
from keras.layers import Input, Dense, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D,Activation
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model


import keras.backend as K
K.set_image_data_format('channels_last')

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

%matplotlib inline

In [5]:
train_set_x,train_set_y,test_set_x,test_set_y,classes = load_dataset()
X_train = train_set_x / 255
X_test = test_set_x / 255

Y_train = train_set_y.reshape(600,1)
Y_test = test_set_y.reshape(150,1)

#print('Number of training examples are = ' + str(X_train[0]))
#print('Number of test examples are = ' + str(X_train[0]))
print('X_train shape = ' + str(X_train.shape))
print('X_test shape = ' + str(X_test.shape))
print('Y_train shape = ' + str(Y_train.shape))
print('Y_test shape = ' + str(Y_test.shape))



X_train shape = (600, 64, 64, 3)
X_test shape = (150, 64, 64, 3)
Y_train shape = (600, 1)
Y_test shape = (150, 1)


In [6]:
def model(input_shape):
    X_input  = Input(input_shape)
    
    X = ZeroPadding2D((3,3))(X_input)
    X = Conv2D(32,(7,7),strides=(1,1),name = 'conv0')(x)
    X = BatchNormalization(axis=3,name='bn0')(x)
    X= Activation('relu')(X)
    
    X = MaxPool2D((2,2),name = 'max_pool')(X)
    
    X= Flatten()(X)
    X = Dense(1, activation='sigmoid',name = 'fc')(X)
    
    model = Model(input = X_input, outputs = X, name = 'IOTFACE')
    
    return model

In [7]:
def IOTFACE(input_shape):
    
    X_input  = Input(input_shape)
    
    X = ZeroPadding2D((3,3))(X_input)
    X = Conv2D(32,(7,7),strides=(1,1),name = 'conv0')(X)
    X = BatchNormalization(axis=3,name='bn0')(X)
    X= Activation('relu')(X)
    
    X = MaxPooling2D((2,2),name = 'max_pool')(X)
    
    X= Flatten()(X)
    X = Dense(1, activation='sigmoid',name = 'fc')(X)
    
    model = Model(input = X_input, outputs = X, name = 'IOTFACE')
    
    return model

In [8]:
iotface = IOTFACE(X_train.shape[1:])

C:\Users\DMTewari\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("fc..., inputs=Tensor("in..., name="IOTFACE")`
  from ipykernel import kernelapp as app


In [9]:
iotface.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [10]:
iotface.fit(X_train, Y_train, epochs = 40,batch_size =16)

Epoch 1/40
600/600 [==============================] - 16s 27ms/step - loss: 1.3250 - acc: 0.6700
Epoch 2/40
600/600 [==============================] - 15s 26ms/step - loss: 0.2999 - acc: 0.8867
Epoch 3/40
600/600 [==============================] - 15s 25ms/step - loss: 0.1823 - acc: 0.9350
Epoch 4/40
600/600 [==============================] - 12s 20ms/step - loss: 0.0868 - acc: 0.9633
Epoch 5/40
600/600 [==============================] - 10s 17ms/step - loss: 0.0948 - acc: 0.9717
Epoch 6/40
600/600 [==============================] - 9s 15ms/step - loss: 0.2048 - acc: 0.9150
Epoch 7/40
600/600 [==============================] - 8s 13ms/step - loss: 0.0961 - acc: 0.9683
Epoch 8/40
600/600 [==============================] - 8s 13ms/step - loss: 0.2182 - acc: 0.9283
Epoch 9/40
600/600 [==============================] - 8s 13ms/step - loss: 0.0815 - acc: 0.9750
Epoch 10/40
600/600 [==============================] - 8s 13ms/step - loss: 0.0738 - acc: 0.9833
Epoch 11/40
600/600 [=============

In [11]:
preds = iotface.evaluate(X_test,Y_test,batch_size=32,verbose=1, sample_weight=None)

print()
print('loss =' +str(preds[0]))
print('accuracy =' +str(preds[1]))

150/150 [==============================] - 2s 15ms/step

loss =0.2701105793317159
accuracy =0.9266666706403096


In [12]:
iotface.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 70, 70, 3)         0         
_________________________________________________________________
conv0 (Conv2D)               (None, 64, 64, 32)        4736      
_________________________________________________________________
bn0 (BatchNormalization)     (None, 64, 64, 32)        128       
_________________________________________________________________
activation_1 (Activation)    (None, 64, 64, 32)        0         
_________________________________________________________________
max_pool (MaxPooling2D)      (None, 32, 32, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 32768)             0         
__________

In [17]:
!pip install graphviz

In [18]:
plot_model(iotface, to_file='oitface.png')

OSError: `pydot` failed to call GraphViz.Please install GraphViz (https://www.graphviz.org/) and ensure that its executables are in the $PATH.